### 1\. O que são Blueprints?

Blueprints, ou "plantas", são componentes que permitem organizar uma aplicação Flask em múltiplos "mini-aplicativos" modulares. Cada Blueprint pode ter suas próprias rotas, templates, arquivos estáticos e lógica de negócio. Eles não são aplicações independentes, mas precisam ser registrados em uma aplicação Flask principal para se tornarem ativos.

Usar Blueprints torna sua aplicação mais:

  * **Modular**: O código é dividido em componentes lógicos.
  * **Reutilizável**: Componentes podem ser facilmente reutilizados em diferentes projetos.
  * **Organizada**: A estrutura do projeto fica mais clara, com cada funcionalidade em seu próprio diretório.
  * **Gerenciável**: A manutenção se torna mais simples, pois cada parte do código está em seu lugar.

-----

### 2\. Estrutura de Projeto Completa

Para este tutorial, a estrutura do seu projeto deve ser a seguinte. Crie as pastas e os arquivos exatamente como mostrado abaixo.

```
/meu_projeto
|-- run.py              # Script para iniciar a aplicação
|-- /app                # Onde a aplicação principal é criada e os blueprints são registrados
|   |-- __init__.py     # Criação da aplicação principal
|   |-- /auth           # Módulo para autenticação
|   |   |-- __init__.py     # Criação do Blueprint 'auth'
|   |   |-- routes.py       # Rotas de login, registro, logout, etc.
|   |   |-- /templates/auth/ # Templates específicos para o módulo
|   |   |   |-- login.html
|   |   |   |-- registro.html
|   |-- /main           # Módulo principal da aplicação
|   |   |-- __init__.py     # Criação do Blueprint 'main'
|   |   |-- routes.py       # Rotas principais (home, sobre, etc.)
|   |   |-- /templates/main/ # Templates específicos para o módulo
|   |   |   |-- home.html
|   |-- /templates      # Templates globais (como o layout.html)
|   |   |-- layout.html
|-- requirements.txt
```

-----

### 3\. Criando e Registrando os Blueprints

Vamos criar e registrar os Blueprints para a nossa aplicação.

#### Passo 1: O Script de Execução (`run.py`)

Este será o único arquivo que você executará para iniciar sua aplicação.

**`run.py`**

In [ ]:
from app import create_app

app = create_app()

if __name__ == '__main__':
    app.run(debug=True)

#### Passo 2: A Aplicação Principal (`app/__init__.py`)

Este arquivo centraliza a criação da aplicação Flask e o registro de todos os Blueprints.

**`app/__init__.py`**

In [ ]:
from flask import Flask

def create_app():
    app = Flask(__name__)
    app.config['SECRET_KEY'] = 'uma-chave-secreta-forte'

    # Importa e registra o Blueprint 'main'
    from .main import main as main_blueprint
    app.register_blueprint(main_blueprint)

    # Importa e registra o Blueprint 'auth' com um prefixo de URL
    from .auth import auth as auth_blueprint
    app.register_blueprint(auth_blueprint, url_prefix='/auth')

    return app

#### Passo 3: O Blueprint de Autenticação (`app/auth/__init__.py`)

Cria a instância do Blueprint para o módulo de autenticação.

**`app/auth/__init__.py`**

In [ ]:
from flask import Blueprint

# Cria uma instância do Blueprint com o nome 'auth'
# O 'template_folder' informa onde o Flask deve procurar os templates
auth = Blueprint('auth', __name__, template_folder='templates')

from . import routes

#### Passo 4: As Rotas de Autenticação (`app/auth/routes.py`)

Define as rotas de login e registro, usando o objeto `auth`.

**`app/auth/routes.py`**

In [ ]:
from flask import render_template
from . import auth

@auth.route('/login')
def login():
    return render_template('auth/login.html')

@auth.route('/registro')
def registro():
    return render_template('auth/registro.html')

#### Passo 5: O Blueprint Principal (`app/main/__init__.py`)

Cria a instância do Blueprint para o módulo principal da aplicação.

**`app/main/__init__.py`**

In [ ]:
from flask import Blueprint

# Cria uma instância do Blueprint com o nome 'main'
main = Blueprint('main', __name__, template_folder='templates')

from . import routes

#### Passo 6: As Rotas Principais (`app/main/routes.py`)

Define as rotas principais como 'home' e 'sobre', usando o objeto `main`.

**`app/main/routes.py`**

In [ ]:
from flask import render_template
from . import main

@main.route('/')
@main.route('/home')
def home():
    return render_template('main/home.html')

@main.route('/sobre')
def sobre():
    return 'Página sobre'

-----

### 4\. Criando os Templates HTML

Agora, vamos criar os templates que serão renderizados pelas rotas. A herança de templates garante que o layout seja consistente.

#### O Template Global (`app/templates/layout.html`)

Este é o template base que todos os outros templates estenderão.

**`app/templates/layout.html`**

```html
<!DOCTYPE html>
<html lang="pt-br">
<head>
    <title>{% block title %}Meu Site{% endblock %}</title>
</head>
<body>
    <header>
        <nav>
            <a href="{{ url_for('main.home') }}">Home</a>
            <a href="{{ url_for('auth.login') }}">Login</a>
            <a href="{{ url_for('auth.registro') }}">Registro</a>
        </nav>
    </header>
    <main>
        {% block content %}{% endblock %}
    </main>
</body>
</html>
```

#### O Template de Login (`app/auth/templates/auth/login.html`)

Este template é específico do Blueprint `auth`.

**`app/auth/templates/auth/login.html`**

```html
{% extends "layout.html" %}

{% block title %}Login{% endblock %}

{% block content %}
    <h2>Página de Login</h2>
    <p>Este template está no Blueprint 'auth'.</p>
{% endblock %}
```

#### O Template de Registro (`app/auth/templates/auth/registro.html`)

Este template também pertence ao Blueprint `auth`.

**`app/auth/templates/auth/registro.html`**

```html
{% extends "layout.html" %}

{% block title %}Registro{% endblock %}

{% block content %}
    <h2>Página de Registro</h2>
    <p>Este template também está no Blueprint 'auth'.</p>
{% endblock %}
```

#### O Template da Home (`app/main/templates/main/home.html`)

Este template pertence ao Blueprint `main`.

**`app/main/templates/main/home.html`**

```html
{% extends "layout.html" %}

{% block title %}Home{% endblock %}

{% block content %}
    <h2>Página Inicial</h2>
    <p>Este template está no Blueprint 'main'.</p>
{% endblock %}
```

-----

### 5\. Executando a Aplicação

Com a estrutura completa, você pode executar o `run.py` no terminal:

```bash
python run.py
```

Sua aplicação estará rodando. Ao navegar para `http://127.0.0.1:5000/`, você verá a página inicial. As rotas `/auth/login` e `/auth/registro` também estarão funcionando, demonstrando como os Blueprints organizam e prefixam as rotas de forma lógica.